In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from scipy.stats import zscore

# Load dataset
df = pd.read_csv("breast-cancer.csv")
print(df.isnull().sum())
df.shape

id                         0
diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64


(569, 32)

In [2]:
df.shape

(569, 32)

In [3]:
# i. Data Cleaning
df.replace('?', np.nan, inplace=True)
df.dropna(inplace=True)

# Remove negative values (if any)
df = df[(df.select_dtypes(include='number') >= 0).all(axis=1)]

print(df.isnull().sum())

df.head()

id                         0
diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
df.shape

(569, 32)

In [5]:
# Exclude 'diagnosis' column for outlier detection
features = df.drop('diagnosis', axis=1)

# Compute z-scores and filter out outliers
z_scores = np.abs(zscore(features))
df = df[(z_scores < 3).all(axis=1)]
df.shape

(487, 32)

In [6]:
# Encode 'diagnosis' column: B = 1 (Benign), M = 0 (Malignant)
le = LabelEncoder()
df['target'] = le.fit_transform(df['diagnosis'])  # M=1, B=0 (default)


In [7]:
# Drop original label and define features and labels
X = df.drop(['diagnosis', 'target'], axis=1)
y = df['target']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Logistic Regression Model
lr = LogisticRegression(max_iter=200)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=200)

In [9]:
lr_pred = lr.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred) * 100

print(f"Logistic Regression Accuracy: {lr_acc:.2f}%")

Logistic Regression Accuracy: 98.98%


In [10]:
nb = GaussianNB()
nb.fit(X_train, y_train)

GaussianNB()

In [11]:
nb_pred = nb.predict(X_test)
nb_acc = accuracy_score(y_test, nb_pred) * 100
print(f"Naïve Bayes Accuracy: {nb_acc:.2f}%")

Naïve Bayes Accuracy: 94.90%


In [12]:
# Comparison Output
print("=== Breast Cancer Tumor Classification ===")
print(f"Regression Method (Logistic Regression) Accuracy: {lr_acc:.2f}%")
print(f"Naive Bayes Method Accuracy: {nb_acc:.2f}%")

if lr_acc > nb_acc:
    print("→ Regression method performed better.")
elif nb_acc > lr_acc:
    print("→ Naive Bayes method performed better.")
else:
    print("→ Both models performed equally well.")


=== Breast Cancer Tumor Classification ===
Regression Method (Logistic Regression) Accuracy: 98.98%
Naive Bayes Method Accuracy: 94.90%
→ Regression method performed better.
